# SET-UP

In [55]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [56]:
def read_file(path):
    with open(path) as f:
        content = f.readlines()
    return content

In [57]:
! ls ../data

emerging.test  wnut17train.conll


In [58]:
wnut_path = "../data/wnut17train.conll"

In [77]:
test_path = "../data/emerging.test.annotated"

In [60]:
import pandas as pd
df = pd.read_csv("../data/wnut17train.conll",sep="\t")

In [61]:
import pandas as pd
df_test = pd.read_csv(test_path,sep="\t")

In [62]:
def get_labels(dataframe):
    labels = []
    for i,row in df.iterrows():
        if(row.O != 'O' and (not isinstance(row.O,float))):
            labels.append(row.O)
    labels = [x[2:len(x)] for x in labels]
    labels = set(labels)

In [63]:
labels = get_labels(df)

In [64]:
def get_words(path):
    content = [x.replace("\t", " ") for x in read_file(path)]
    text = [x.split() for x in content]
    return text

In [78]:
text_train = get_words(wnut_path)
text_test = get_words(test_path)
datasets = [text_train,text_test]

In [79]:
text_test[0:10]

[['&', 'O'],
 ['gt', 'O'],
 [';', 'O'],
 ['*', 'O'],
 ['The', 'O'],
 ['soldier', 'O'],
 ['was', 'O'],
 ['killed', 'O'],
 ['when', 'O'],
 ['another', 'O']]

In [80]:
def squash(list_labels):
    res = []
    for i,x in enumerate(list_labels):
        j = 1
        if(i + j< len(list_labels) and x[1] == 'B' and list_labels[i  + j][1] == 'I'):
            while(i + j < len(list_labels) and list_labels[i + j][1] == 'I'):
                j += 1
            res.append((x[0],x[0] + j -1 ,x[2]))
        elif(x[1] == 'B'):
            res.append([x[0],x[2]])
    return res

In [81]:
def str_i_to_char_i(sent_label):
    res_labels = []
    sent = sent_label[0].split()
    labels = sent_label[1]
    
    for label in labels: 
        if(len(label) == 2):
            start_label_index = label[0]
            end_label_index = label[0]
            label_name = label[1]
        elif(len(label) == 3):
            start_label_index = label[0]
            end_label_index = label[1]
            label_name = label[2]
            


        start_char_index = sum([len(x) for x in sent[0:start_label_index]])    
        end_char_index = start_char_index + sum([len(x) for x in sent[start_label_index:end_label_index + 1]])
        res_labels.append((start_char_index + start_label_index,end_char_index + end_label_index,label_name))
        
            
    return [" ".join(sent), res_labels]

In [82]:
def to_spacy_format(sent_label): 
    sent = sent_label[0]
    labels = sent_label[1]
    res_dict = {'entities': labels}
    return [sent, res_dict]

In [83]:
import nltk
from nltk.tokenize import sent_tokenize 
def create_entities_char_level(text):
    res = []
    sentence = []
    sent_labels = []
    for word_entity_pair in text:
        if(len(word_entity_pair)!=0):
            sentence.append(word_entity_pair[0])
            sent_labels.append(word_entity_pair[1])
        else:
            sent_labels = [(i,x[0], x[2:len(x)]) for i,x in enumerate(sent_labels) if x != 'O' ]
            sent_labels = squash(sent_labels)
            res.append([" ".join(sentence),sent_labels])
            sent_labels = []
            sentence = []
        
    res = [str_i_to_char_i(x) for x in res]
    res = [to_spacy_format(x) for x in res]
    return res

In [84]:
TRAIN_DATA = create_entities_char_level(text_train)
EVAL_DATA = create_entities_char_level(text_test)

In [85]:
TRAIN_DATA[1]

['From Green Newsfeed : AHFA extends deadline for Sage Award to Nov . 5 http://tinyurl.com/24agj38',
 {'entities': [(22, 26, 'group')]}]

In [86]:
EVAL_DATA[0]

['& gt ; * The soldier was killed when another avalanche hit an army barracks in the northern area of Sonmarg , said a military spokesman .',
 {'entities': [(100, 107, 'location')]}]

### TRAIN

In [92]:
import random
import spacy
model = None
n_iter=35

if model is not None:
    nlp1 = spacy.load(model)  # load existing spaCy model
    print("Loaded model '%s'" % model)
else:
    nlp1 = spacy.blank('en')  # create blank Language class
    print("Created blank 'en' model")

#create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp1.pipe_names:
    print("Add ner pipe")
    ner = nlp1.create_pipe('ner')
    nlp1.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = nlp1.get_pipe('ner')


Created blank 'en' model
Add ner pipe


In [93]:
# add labels, Trains data based on annotations 
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

In [ ]:
from tqdm import tqdm
# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp1.pipe_names if pipe != 'ner']

with nlp1.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp1.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            nlp1.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=0.5,  # dropout 
                sgd=optimizer,  # callable to update weights
                losses=losses)
        print(losses)

  0%|          | 4/3394 [00:00<01:52, 30.22it/s]

{'ner': 4858.2233301231045}


  0%|          | 3/3394 [00:00<02:12, 25.65it/s]

{'ner': 4484.49461176307}


  0%|          | 3/3394 [00:00<01:53, 29.99it/s]

...

  0%|          | 2/3394 [00:00<03:15, 17.31it/s]

{'ner': 4198.263415937665}


  0%|          | 2/3394 [00:00<02:53, 19.53it/s]

{'ner': 3976.363810165486}


 74%|███████▍  | 2518/3394 [01:28<00:27, 31.56it/s]

# EVALUATION 

In [91]:
# test the trained model
for text,entities in EVAL_DATA:
    doc = nlp1(text)
    print(entities.values())
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

dict_values([[(100, 107, 'location')]])
Entities []
dict_values([[(54, 67, 'location')]])
Entities []
dict_values([[]])
Entities []
dict_values([[]])
Entities []
dict_values([[(77, 99, 'group'), (102, 105, 'group')]])
Entities []
dict_values([[(161, 181, 'person')]])
Entities []
dict_values([[(33, 45, 'location')]])
Entities [('Gurez', 'location')]
dict_values([[(11, 14, 'group'), (50, 62, 'location')]])
Entities [('Gurez', 'location')]
dict_values([[(11, 14, 'group')]])
Entities []
dict_values([[(164, 176, 'person')]])
Entities []
dict_values([[(6, 30, 'creative-work')]])
Entities [('Making', 'location')]
dict_values([[(83, 92, 'location')]])
Entities []
dict_values([[(99, 103, 'group')]])
Entities []
dict_values([[]])
Entities []
dict_values([[]])
Entities []
dict_values([[(33, 41, 'location')]])
Entities []
dict_values([[]])
Entities []
dict_values([[(4, 8, 'group')]])
Entities []
dict_values([[(19, 23, 'group')]])
Entities []
dict_values([[]])
Entities []
dict_values([[(4, 45, 'gro

dict_values([[(18, 30, 'person'), (39, 53, 'group')]])
Entities [('Andy Carroll', 'person'), ('Crystal Palace', 'person')]
dict_values([[]])
Entities []
dict_values([[]])
Entities []
dict_values([[(2, 16, 'person')]])
Entities []
dict_values([[]])
Entities []
dict_values([[]])
Entities []
dict_values([[(65, 70, 'person')]])
Entities []
dict_values([[(4, 20, 'person')]])
Entities []
dict_values([[(27, 40, 'group'), (58, 72, 'person')]])
Entities []
dict_values([[]])
Entities []
dict_values([[(7, 16, 'group'), (48, 56, 'person'), (59, 65, 'person'), (70, 78, 'person')]])
Entities []
dict_values([[(2, 14, 'person')]])
Entities []
dict_values([[(33, 58, 'corporation')]])
Entities []
dict_values([[]])
Entities []
dict_values([[(38, 41, 'group'), (47, 61, 'person')]])
Entities []
dict_values([[]])
Entities []
dict_values([[]])
Entities []
dict_values([[(40, 49, 'person')]])
Entities []
dict_values([[(27, 29, 'corporation')]])
Entities []
dict_values([[(16, 30, 'creative-work')]])
Entities []

dict_values([[(59, 62, 'person')]])
Entities []
dict_values([[(47, 51, 'product')]])
Entities []
dict_values([[]])
Entities []
dict_values([[(0, 16, 'group'), (97, 107, 'group'), (125, 136, 'group'), (182, 191, 'group'), (228, 266, 'group'), (310, 319, 'group')]])
Entities []
dict_values([[(26, 33, 'creative-work')]])
Entities []
dict_values([[(8, 15, 'person'), (35, 42, 'creative-work')]])
Entities []
dict_values([[(28, 38, 'group'), (78, 88, 'group')]])
Entities [('Parliament', 'location')]
dict_values([[(21, 28, 'product')]])
Entities []
dict_values([[]])
Entities []
dict_values([[]])
Entities []
dict_values([[(50, 54, 'person'), (80, 89, 'location'), (148, 158, 'location')]])
Entities []
dict_values([[(46, 68, 'product'), (73, 93, 'product')]])
Entities []
dict_values([[(3, 9, 'person'), (29, 39, 'location')]])
Entities []
dict_values([[(43, 46, 'product')]])
Entities []
dict_values([[(0, 22, 'product')]])
Entities []
dict_values([[(52, 66, 'location'), (119, 124, 'person'), (210, 

dict_values([[]])
Entities []
dict_values([[(2, 8, 'person')]])
Entities []
dict_values([[]])
Entities []
dict_values([[(2, 10, 'person')]])
Entities []
dict_values([[(57, 61, 'group')]])
Entities []
dict_values([[]])
Entities []
dict_values([[(16, 27, 'product'), (53, 65, 'group')]])
Entities []
dict_values([[]])
Entities []
dict_values([[(63, 88, 'product')]])
Entities []
dict_values([[]])
Entities []
dict_values([[]])
Entities []
dict_values([[(0, 10, 'creative-work'), (21, 39, 'person'), (59, 62, 'person')]])
Entities []
dict_values([[(25, 34, 'person'), (157, 164, 'person')]])
Entities []
dict_values([[]])
Entities []
dict_values([[]])
Entities []
dict_values([[]])
Entities []
dict_values([[(40, 48, 'creative-work')]])
Entities []
dict_values([[]])
Entities []
dict_values([[(30, 42, 'creative-work')]])
Entities []
dict_values([[(26, 28, 'person'), (50, 60, 'product')]])
Entities []
dict_values([[]])
Entities []
dict_values([[]])
Entities []
dict_values([[]])
Entities []
dict_value

dict_values([[(0, 3, 'location'), (12, 25, 'person')]])
Entities []
dict_values([[]])
Entities []
dict_values([[]])
Entities [('Umple', 'location')]
dict_values([[]])
Entities []
dict_values([[]])
Entities []
dict_values([[(3, 7, 'person'), (70, 74, 'person')]])
Entities []
dict_values([[(14, 20, 'person'), (23, 29, 'person'), (45, 51, 'person'), (84, 90, 'person'), (120, 126, 'person')]])
Entities []
dict_values([[]])
Entities []
dict_values([[(5, 16, 'person')]])
Entities []
dict_values([[(5, 16, 'person')]])
Entities []
dict_values([[]])
Entities []
dict_values([[]])
Entities []
dict_values([[]])
Entities []
dict_values([[]])
Entities []
dict_values([[(32, 39, 'corporation'), (53, 62, 'product')]])
Entities [('Prompto', 'location')]
dict_values([[]])
Entities []
dict_values([[]])
Entities []
dict_values([[(0, 25, 'creative-work')]])
Entities []
dict_values([[(27, 33, 'product')]])
Entities []
dict_values([[(0, 9, 'location')]])
Entities []
dict_values([[(96, 102, 'product')]])
Entit

dict_values([[(3, 7, 'product')]])
Entities []
dict_values([[(2, 11, 'person')]])
Entities []
dict_values([[(5, 21, 'person')]])
Entities []
dict_values([[(26, 37, 'location')]])
Entities [('Golden Eagle', 'person')]
dict_values([[(2, 10, 'person'), (13, 18, 'person')]])
Entities []
dict_values([[(71, 73, 'person')]])
Entities []
dict_values([[]])
Entities []
dict_values([[]])
Entities []
dict_values([[]])
Entities []
dict_values([[(5, 18, 'person'), (51, 63, 'creative-work')]])
Entities []
dict_values([[(9, 30, 'corporation'), (82, 90, 'corporation')]])
Entities []
dict_values([[]])
Entities []
dict_values([[]])
Entities []
dict_values([[(35, 49, 'person'), (58, 64, 'person')]])
Entities []
dict_values([[]])
Entities []
dict_values([[]])
Entities []
dict_values([[]])
Entities []
dict_values([[]])
Entities []
dict_values([[(2, 14, 'group'), (36, 50, 'creative-work')]])
Entities []
dict_values([[(5, 16, 'person')]])
Entities []
dict_values([[]])
Entities []
dict_values([[(28, 31, 'perso

dict_values([[]])
Entities [('Anyone', 'group')]
dict_values([[(0, 13, 'person')]])
Entities []
dict_values([[]])
Entities []
dict_values([[]])
Entities [('Strategic Relationship', 'person')]
dict_values([[(2, 17, 'group'), (50, 54, 'corporation')]])
Entities []
dict_values([[(5, 16, 'creative-work')]])
Entities []
dict_values([[(10, 15, 'person')]])
Entities []
dict_values([[(5, 17, 'person')]])
Entities []
dict_values([[(2, 8, 'person'), (15, 24, 'creative-work'), (41, 44, 'corporation'), (73, 83, 'person')]])
Entities []
dict_values([[(2, 8, 'person')]])
Entities []
dict_values([[(2, 14, 'person'), (17, 27, 'corporation'), (30, 38, 'person')]])
Entities []
dict_values([[(17, 36, 'creative-work'), (41, 48, 'creative-work')]])
Entities [('KODE Action', 'person')]
dict_values([[]])
Entities []
dict_values([[(0, 7, 'location'), (63, 70, 'location')]])
Entities [('Ireland', 'location')]
dict_values([[(0, 13, 'group')]])
Entities []
dict_values([[]])
Entities [('Caupain', 'location')]
dic

dict_values([[(5, 15, 'person'), (137, 144, 'product')]])
Entities []
dict_values([[]])
Entities []
dict_values([[(0, 14, 'person')]])
Entities []
dict_values([[]])
Entities []
dict_values([[(5, 10, 'person'), (13, 29, 'person')]])
Entities [('Kellyanne Conway', 'person')]
dict_values([[(61, 71, 'person')]])
Entities []
dict_values([[]])
Entities []
dict_values([[(62, 73, 'location')]])
Entities []
dict_values([[]])
Entities []
dict_values([[]])
Entities []
dict_values([[]])
Entities []
dict_values([[]])
Entities []
dict_values([[(14, 20, 'location'), (23, 29, 'location'), (65, 70, 'location'), (71, 76, 'location')]])
Entities []
dict_values([[(5, 12, 'person'), (82, 92, 'person')]])
Entities []
dict_values([[]])
Entities []
dict_values([[(2, 11, 'person'), (33, 40, 'creative-work')]])
Entities []
dict_values([[(0, 10, 'person')]])
Entities []
dict_values([[]])
Entities []
dict_values([[(5, 19, 'group'), (59, 63, 'group'), (140, 147, 'group')]])
Entities []
dict_values([[]])
Entities [

In [18]:
TRAIN_DATA[0:10]

[['#ForDrinkersOnly Fridays Brings You #JOEGIFTED &amp; #BIGBEN LIVE THIS Friday at #ClubLacura !! http://t.co/TicD9UibSQ 6',
  {'entities': [(36, 46, 'person'),
    (53, 60, 'person'),
    (81, 92, 'location')]}],
 ["i'm definitely eating tacos for dinner . the update for green may or may not have had something to do with it ...",
  {'entities': []}],
 ["Come enjoy a movie in our romantic theater room . January is 25% off and is almost over , don't miss out ! Book ... http://t.co/qwYDk4dXI6",
  {'entities': []}],
 ["... four fucking years I 've been configuring iBGP and eBGP on Cisco AND Juniper routers for America 's largest wireless ISP ... and I get a 612",
  {'entities': [(63, 68, 'corporation'),
    (73, 80, 'corporation'),
    (93, 100, 'location')]}],
 ['RT @IlliniCampusRec : Orange &amp; Blue Skate tonight , 7:30 - 9:30 pm ! Free for UI faculty/staff &amp; their immediate families . http://bit.ly/c1M8dr',
  {'entities': [(82, 84, 'group')]}],
 ['JANUARY 3 pwede na ulit magship

In [ ]:
t